# 5 days data cleaning challenge

I was on kaggle searching new dataset for my next project, and randomly came across this interesting challenge that I want to give it a try. This challenge consists of the following:

* [Day 1: Handling missing values](https://www.kaggle.com/rtatman/data-cleaning-challenge-handling-missing-values)
* [Day 2: Scaling and normalization](https://www.kaggle.com/rtatman/data-cleaning-challenge-scale-and-normalize-data)
* [Day 3: Parsing dates](https://www.kaggle.com/rtatman/data-cleaning-challenge-parsing-dates/)
* [Day 4: Character encodings](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings/)
* [Day 5: Inconsistent Data Entry](https://www.kaggle.com/rtatman/data-cleaning-challenge-inconsistent-data-entry/)

The creator used American Football games event for demonstration, and we will be using a dataset of building permits issued in San Francisco for the exercise. As for Day 1, it will be foscusing on "Handling missing values", and here are the tasks for the first challenge:

1.  [Take a first look at the data](##Take-a-first-look-at-the-data)
2.  [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
3.  [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
4.  [Drop missing values](#Drop-missing-values)
5.  [Filling in missing values](#Filling-in-missing-values)

Let's get this started!

# Import libraries and dataset

In [1]:
# import libraries
import numpy as np
import pandas as pd

# 1. Take a first look at the data
________
Below is an overview of the dataframe showing the first 5 rows, I am looking to see the details of the dataframe to get an idea of what's going on with the data, including the data type of each column and total of non-null data.

In [2]:
# load dataset
df = pd.read_csv('/Users/queeniehu/anaconda3/Training/Projects/mini_projects/02_5_day_challenge/data/Building_Permits.csv')
df.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


In [3]:
df.shape

(198900, 43)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 43 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Permit Number                           198900 non-null  object 
 1   Permit Type                             198900 non-null  int64  
 2   Permit Type Definition                  198900 non-null  object 
 3   Permit Creation Date                    198900 non-null  object 
 4   Block                                   198900 non-null  object 
 5   Lot                                     198900 non-null  object 
 6   Street Number                           198900 non-null  int64  
 7   Street Number Suffix                    2216 non-null    object 
 8   Street Name                             198900 non-null  object 
 9   Street Suffix                           196132 non-null  object 
 10  Unit                                    2947

# 2. See how many missing data points we have
________
Using the syntax `isnull` (or `isna`), we could see how many missing data observed in each column 

In [9]:
null_count = df.isna().sum()
null_count

Permit Number                                  0
Permit Type                                    0
Permit Type Definition                         0
Permit Creation Date                           0
Block                                          0
Lot                                            0
Street Number                                  0
Street Number Suffix                      196684
Street Name                                    0
Street Suffix                               2768
Unit                                      169421
Unit Suffix                               196939
Description                                  290
Current Status                                 0
Current Status Date                            0
Filed Date                                     0
Issued Date                                14940
Completed Date                            101709
First Construction Document Date           14946
Structural Notification                   191978
Number of Existing S

In [24]:
# how many total missing values do we have?
total_cells = np.product(df.shape)
total_missing = null_count.sum()

# percent of data that is missing
percent_missing_data = (total_missing/total_cells) * 100
print('Percentage of missing data: ' + str(percent_missing_data))

Percentage of missing data: 26.26002315058403


**Out of 43 columns within the DataFrame, it consists of 26.26% of null data**

# 3. Figure out why the data is missing

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

In [40]:
print(df[['Street Number Suffix', 'Zipcode']].sample(10))
#Looks like all values in street number suffix are null
#Lets check with the null counts of the column
(null_count['Street Number Suffix']/df.shape[0])*100

       Street Number Suffix  Zipcode
12234                   NaN  94117.0
135126                  NaN  94104.0
157308                  NaN  94122.0
95716                   NaN  94109.0
36946                   NaN  94102.0
183732                  NaN  94111.0
21763                   NaN  94116.0
198067                  NaN  94104.0
8014                    NaN  94117.0
49451                   NaN  94114.0


98.88587229763701

We can see that this gives us a percentage of 98.88%!! That's a lot of missing values.
Further, looking at the Data Dictionary fo the SF permits data, we can see that - Street Number Suffix is something that is probably missing because it doesn't exist! Whereas, Zipcode is probably missing because it wasn't recorded!

# 4. Drop missing values

In [25]:
# Create a copy of the dataframe
df_sf = df.copy()

In [28]:
df_sf.dropna()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


**If we do a `dropna()` syntax, it will remove all the data. This is possibly because every row in the dataframe had at least one null value. Try to remove only columns that has missing values**

In [29]:
df_sf2 = df.copy()

In [30]:
df_sf2 = df_sf2.dropna(axis=1)
df_sf2.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992


In [32]:
df_sf2.shape

(198900, 12)

**After dopping columns that consist of null data, the data set is now down to 12 columns only.**

# 5. Filling in missing values

**Another option is to use Pandas `fillna()` function to fill in all the missing values within the DataFrame, that way we will retain the full full dataset.**

In [36]:
df_sf_fillna = df.copy()
df_sf_fillna.fillna(0)
df_sf_fillna.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


In [37]:
df_sf_fillna.shape

(198900, 43)